In [1]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()
sess = sagemaker.Session()

# Upload dataset to S3
s3_train_path = sess.upload_data("phishing_data.csv", key_prefix="phishing/train")

# Use SageMaker’s built-in XGBoost container
container = sagemaker.image_uris.retrieve("xgboost", sess.boto_region_name, "1.5-1")

xgb = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path=f"s3://{sess.default_bucket()}/output",
    sagemaker_session=sess
)

xgb.set_hyperparameters(objective="binary:logistic", num_round=100)
xgb.fit({"train": s3_train_path})

# Deploy model
predictor = xgb.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")


ModuleNotFoundError: No module named 'sagemaker'